# Dataset description
* 32 participants [s01 to s32]
* 40 pieces of music
* 40 channels out of which the first 32 are relevant 
* 8064 readings recorded at 128Hz


In [1]:
import datetime

In [2]:
from deap_pre_process import *

In [3]:
eeg1d,eeg2d,val,aro = get_subject_data(subject_no=5)

In [25]:
eeg1d.shape

(2400, 128, 32)

In [24]:
eeg1d[0][0]

array([-0.44553747, -1.30729085,  0.61818628, -1.97475527, -0.76152008,
       -0.23789995,  0.05069117, -1.54751224, -0.41305226,  1.29244011,
        0.32111143, -0.34914847,  0.87825302,  0.87572375,  1.49782983,
        0.60688355, -0.0739179 , -0.07013766, -0.76914595, -0.61642409,
       -1.0477853 , -1.61118978,  0.79399241,  1.71433644, -0.83146426,
       -0.8493887 , -0.81053115,  1.4603206 ,  1.19104473,  0.10162078,
        1.10004079,  1.2142265 ])

# Making the dummy network


In [5]:
from rnn_function import *
from cnn_function import *
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13074696698980798675
]


In [7]:
#Constant :
WINDOW_SIZE = 128    #DO NOT CHANGE
#For rnn layers
INPUT_NODES = 32
HIDDEN_ST = 32
HOLD_PROBA = 0.5
RNN_INPUT_SIZE = 32

#For cnn layers
CONV_IP_SHAPE = [-1,9,9,1]
CONV1_KERNEL = [4,4,1,32]
CONV2_KERNEL = [4,4,32,64]
CONV3_KERNEL = [4,4,64,128]
REDUCTION_KERNEL = [1,1,128*WINDOW_SIZE,13]

#For dnn layers (for the paper code)
N_FC_IN = 1024
N_FC_OUT= 1024

#FOR PREDICTIONS
N_LABELS = 2
LEARNING_RATE = 0.0001

In [8]:
#FUNCTION FORM https://github.com/ynulonger/ijcnn/blob/master/cv.py
#FOR SHAPING THE INPUT OF LSTM
def apply_fully_connect(x, x_size, fc_size):
    fc_weight = init_weight([x_size, fc_size])
    fc_bias = init_bias([fc_size])
    return tf.nn.elu(tf.add(tf.matmul(x, fc_weight), fc_bias))

#FOR READOUT OF THE FINAL LAYER
def apply_readout(x, x_size, readout_size):
    readout_weight = init_weight([x_size, readout_size])
    readout_bias = init_bias([readout_size])
    return tf.add(tf.matmul(x, readout_weight), readout_bias)

In [9]:
#Resetting tf default graph
tf.reset_default_graph()

#CNN placeholders and model variables
x_data_cnn = tf.placeholder(dtype = tf.float32,shape=(None,9,9),name='cnn_data')
x_image = tf.reshape(x_data_cnn,CONV_IP_SHAPE,name='cnn_image')

#LAYERS
conv1 = conv_layer(x_image,CONV1_KERNEL,name="convo_l1")
conv2 = conv_layer(conv1,CONV2_KERNEL,name="convo_l2")
conv3 = conv_layer(conv2,CONV3_KERNEL,name="convo_l3")

#Depth variable
depth = tf.reshape(conv3,[-1,9,9,128* WINDOW_SIZE])

#Size reduction conv layer
reduced_data = conv_layer(depth,REDUCTION_KERNEL,name="conv_reduced")

#Flattening convolution layer
shape = reduced_data.get_shape().as_list()
final_flat = tf.reshape(reduced_data, [-1, shape[1] * shape[2] * shape[3]])
cnn_out_fuse = final_flat

#RNN placeholder and model variables
x_data_rnn = tf.placeholder(dtype = tf.float32,shape=(None,WINDOW_SIZE,INPUT_NODES),name='rnn_data') #TIME_STEP = WINDOW_SIZE
keep_prob = tf.placeholder(dtype=tf.float32)

#Shaping the RNN input
shape = x_data_rnn.get_shape().as_list()
rnn_in_flat = tf.reshape(x_data_rnn, [-1, shape[2]]) #[batch_size*n_time_step, n_electrode]
rnn_fc_in = apply_fully_connect(rnn_in_flat, shape[2], N_FC_IN) #[batch_size*n_time_step, n_electrode]
lstm_in = tf.reshape(rnn_fc_in, [-1,WINDOW_SIZE,N_FC_IN]) #[batch_size, n_time_step, n_fc_in]

#Making cells
cell1 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cell2 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cells = [cell1,cell2]
final_cell = create_Stacked_cell(cells)

#Laying out the network
output, states = create_RNN(final_cell,lstm_in)

#Unstacking the output
output = tf.unstack(tf.transpose(output, [1, 0, 2]), name='lstm_out')

#Selecting the final output form the layer
rnn_output = output[-1] #[batch, fc_size]

#Shaping the output
lstm_fc_out = dnn_layer(rnn_output,N_FC_OUT) #[batch_size, n_fc_out]
lstm_fc_drop = tf.nn.dropout(lstm_fc_out, keep_prob)

#Placeholder for true values
y_true = tf.placeholder(dtype=tf.float32,shape=(None,N_LABELS)) #[low, high]

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [10]:
#Flattening the layer
final_connected_layer = tf.concat([cnn_out_fuse, lstm_fc_drop], axis=1)
shape = final_connected_layer.get_shape().as_list()
print("\nfuse_cnn_rnn:", shape)


fuse_cnn_rnn: [None, 2077]


In [11]:
#Creating output variables 
y_ = apply_readout(final_connected_layer, shape[1], N_LABELS)   #(1,2)
y_pred = tf.argmax(tf.nn.softmax(y_), 1, name="y_pred")
y_posi = tf.nn.softmax(y_, name="y_posi")

#Evaluation values
correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(y_), 1), tf.argmax(y_true, 1))

#Making cost functions
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y_true), name='loss')
train = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
# Saver and variable initializer
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [13]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [50]:
#Training and testing on valance
with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        sub = np.random.randint(1,33)
        print("SUBJECT: ",i)
        eeg1d,eeg2d,val,aro = get_subject_data(subject_no=sub)
        t1 = datetime.datetime.now()
        num_sec=60                 # for total do 40*60 = 2400 seconds
        print("[*] Training",num_sec,"seconds.")
        for k in range(num_sec):
            print('\r',k,'/',num_sec,end='')
            pred = np.zeros((1,2))    # [low_val, high_val]
            pred[0][val[k]]=1
#             pred[0][aro[k]]=1
            x_rnn = eeg1d[k].reshape(-1,WINDOW_SIZE,32)  #(1,128,32)
            x_cnn = eeg2d[k]
            sess.run(train, feed_dict = {keep_prob:0.5,x_data_cnn:x_cnn, x_data_rnn: x_rnn, y_true: pred})
        
        print("\r",num_sec,'/',num_sec)
        t2 = datetime.datetime.now()
        if(i%10 == 0):
            test_s = np.random.randint(1,33)
            test_eeg1d,test_eeg2d,test_val,test_aro = get_subject_data(subject_no=test_s)
            print("\n[+] Calculating accuracy")
            pred = []
            tes_sec = 1000
            for j in range(tes_sec):
                print('\r',j,'/',tes_sec,end='')
                actual = np.zeros((1,2))    #[low_val, high_val]
                actual[0][test_val[j]]=1
                x_rnn = test_eeg1d[j].reshape(-1,WINDOW_SIZE,32)
                x_cnn = test_eeg2d[j]
                ret = sess.run(correct_prediction ,feed_dict = {keep_prob:1,x_data_cnn:x_cnn, x_data_rnn: x_rnn, y_true: actual})
                pred.append(ret)
            accuracy = tf.reduce_mean(tf.cast(np.array(pred), tf.float32), name='accuracy')
            print("\nAccuracy: ", sess.run(accuracy))
            print("\n")
    saver.save("TrainedModel1/model")
        

SUBJECT:  0
[*] Training 60 seconds.
 60 / 60

[+] Calculating accuracy
 999 / 1000
Accuracy:  0.542


SUBJECT:  1
[*] Training 60 seconds.
 60 / 60
SUBJECT:  2
[*] Training 60 seconds.
 60 / 60
SUBJECT:  3
[*] Training 60 seconds.
 60 / 60
SUBJECT:  4
[*] Training 60 seconds.
 60 / 60
SUBJECT:  5
[*] Training 60 seconds.
 60 / 60
SUBJECT:  6
[*] Training 60 seconds.
 60 / 60
SUBJECT:  7
[*] Training 60 seconds.
 60 / 60
SUBJECT:  8
[*] Training 60 seconds.
 60 / 60
SUBJECT:  9
[*] Training 60 seconds.
 60 / 60
SUBJECT:  10
[*] Training 60 seconds.
 60 / 60

[+] Calculating accuracy
 999 / 1000
Accuracy:  0.471


SUBJECT:  11
[*] Training 60 seconds.
 60 / 60
SUBJECT:  12
[*] Training 60 seconds.
 60 / 60
SUBJECT:  13
[*] Training 60 seconds.
 60 / 60
SUBJECT:  14
[*] Training 60 seconds.
 60 / 60
SUBJECT:  15
[*] Training 60 seconds.
 60 / 60
SUBJECT:  16
[*] Training 60 seconds.
 60 / 60
SUBJECT:  17
[*] Training 60 seconds.
 60 / 60
SUBJECT:  18
[*] Training 60 seconds.
 60 / 60
SUB

KeyboardInterrupt: 

In [13]:
print((t2 -t1).seconds," seconds",(t2 -t1).microseconds, "microseconds")

NameError: name 't2' is not defined